# EmbedX: Multi-lingual Embedding Model

## Project Overview

- EmbedX is an Natural Language Processing project focused on developing and comparing different word embedding approaches across multiple languages. 
- The project aims to evaluate the effectiveness of various embedding techniques through practical applications in chatbot development and comprehensive comparative analysis.

## Key Components of this project

- **Part 1.** Multi-lingual Word2Vec Implementation
    - Focus on creating a model to produce Word2Vec embeddings for non-English languages.

- **Part 2.** RNN-based Embedding Architecture 
    - Development of a custom RNN architecture for embedding generation. 
    - Comparative analysis against standard Keras Word2Vec embeddings.

- **Part 3.** Chatbot Integration & Evaluation 
    - Development of an embedding-powered chatbot system.
    - Comparative testing between English and non-English embedding models
    - Performance benchmarking across different embedding sources


- **Part 4.** Embedding Quality Assessment
    - Comparative analysis of Word2Vec and RNN-based approaches
    - Implementation of multiple evaluation metrics


- **Part 5.** DNN Comparative Study
    - Implementation of a shallow DNN for embedding generation
    - Head-to-head comparison with RNN-based embeddings

## Priority Focus
- The core focus is on developing and optimizing the RNN-based embedding architecture, as this forms the foundation for subsequent comparative analyses and applications.

---

## Part 1. Multi-lingual Word2Vec Implementation
- Focus on creating a model to produce Word2Vec embeddings for non-English languages.

### Approach

- Import necessary libraries

In [ ]:
import copy
import pandas as pd
import numpy as np
from string import digits

import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import tensorflow as tf
from gensim.models import Word2Vec
from indicnlp.tokenize import indic_tokenize

- Preprocess the text from the file \
Process the file line by line, remove unwanted characters, and breakdown the content into sentences based on Hindi punctuation

In [2]:
hi_punct = ['!', '?', '|', '||']
sentences = []
with open("dataset/Panchtantra.txt", 'r', encoding="utf-8") as file_data: 
    
    sentence = ''
    paragraph_p = 0
    for l,line in enumerate(file_data): 
        
        # Remove unwanted characters
        line = line.strip() \
                    .replace('।', '') \
                    .replace('॥', '') \
                    .replace('\ufeff', '') \
                    .replace('\u200C', '') \
                    .replace('“', '') \
                    .replace('”', '') \
                    .replace('‘', '') \
                    .replace('’', '') \
                    .replace('—', '') \
                    .replace('-', '') \
                    .replace('$', '')
                    
        
        # One \n is just a carriage return. Two \n's means 
        # a new paragraph, start a new sentence
        if len(line) == 0:
            #print('empty line!')
            paragraph_p += 1
            if paragraph_p == 2:
                if 0 < len(sentence):
                    sentences.append(sentence)
                    sentence = ''
                paragraph_p = 0
            continue
        
        # detect sentence delimeters
        delimeters = []
        for c in hi_punct:
            line_copy = copy.copy(line)
            while -1 < line_copy.find(c):
                d = line_copy.find(c)
                delimeters.append(d)
                line_copy = line_copy[:d] + '_' + line_copy[d+1:]
                    
        delimeters.sort()

            
        # no sentence delimeters: keep appending
        if len(delimeters) == 0:
            sentence += line
            
        # detected sentence delimeter(s), terminate sentence
        else:
                
            last_index = 0
            for i in delimeters:
                sentence += line[last_index : i]
                sentences.append(sentence)

                sentence = ''
                last_index = i + 1
                    
            # at the end of the indexes traversal, add remaining words to next sentence     
            if last_index <= len(line):
                dangling = line[last_index :].strip()
                if 0 < len(dangling):
                    sentence = dangling
                    #print("seeding:", sentence)
                else:
                    sentence = ''
            else:
                sentence = ''


- Check the sentences list

In [3]:
[(i, s) for i,s in enumerate(sentences)]

[(0,
  'पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहानियों का संग्रह है जिसके रचयिता आचार्य विष्णु शर्मा है पंचतंत्र की कहानी में बच्चोंके साथसाथ बड़े भी रुचि लेते हैं पंचतंत्र की कहानी के पीछे कोई ना कोई शिक्षा या मूल छिपा होता है जो हमें सीखदेती है पंचतंत्र की कहानी बच्चे बड़ी चाव से पढ़ते हैं तथा सीख लेते हैं पंचतंत्र की कछ कहानियां ऐसी भी है जोहिंदी में कहानी लेखन में दी जाती हैं तथा इसके साथसाथ कई परीक्षाओं में भी 80780 6 (9 कुछपाठयपृस्तक में दी होती है जो लिखने को आती हैं महत्वपूर्ण और आकर्षक 40 कहानियां इस ब्लॉग में दी गई हैं'),
 (1,
  'आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा पंचतंत्र संस्कत की नीति पर्तक के लेखक माने जाते हैं जब यह ग्रंथ बनकर तैयारहुआ तब विष्ण शर्मा की उम्र 40 सात्र थी विष्ण शर्मा दक्षिण भारत के महिलारोप्य नामक नगर में रहते थे एकराजा के 3 मर्ख पत्र थे जिनकी जिम्मेदारी विष्ण शर्मा को दे दी गई विष्ण शर्मा को यह पता था कि यह इतने मूर्ख हैंकि इनको प्राने तरीकों से नहीं पढ़ाया जा सकता तब उन्होंने जंत कथाओं के द्वारा पढ़ाने का निश्चय कियापंचतंत्र को पांच समूह म

- Preview the first 5 sentences

In [4]:
sentences[:5]

['पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहानियों का संग्रह है जिसके रचयिता आचार्य विष्णु शर्मा है पंचतंत्र की कहानी में बच्चोंके साथसाथ बड़े भी रुचि लेते हैं पंचतंत्र की कहानी के पीछे कोई ना कोई शिक्षा या मूल छिपा होता है जो हमें सीखदेती है पंचतंत्र की कहानी बच्चे बड़ी चाव से पढ़ते हैं तथा सीख लेते हैं पंचतंत्र की कछ कहानियां ऐसी भी है जोहिंदी में कहानी लेखन में दी जाती हैं तथा इसके साथसाथ कई परीक्षाओं में भी 80780 6 (9 कुछपाठयपृस्तक में दी होती है जो लिखने को आती हैं महत्वपूर्ण और आकर्षक 40 कहानियां इस ब्लॉग में दी गई हैं',
 'आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा पंचतंत्र संस्कत की नीति पर्तक के लेखक माने जाते हैं जब यह ग्रंथ बनकर तैयारहुआ तब विष्ण शर्मा की उम्र 40 सात्र थी विष्ण शर्मा दक्षिण भारत के महिलारोप्य नामक नगर में रहते थे एकराजा के 3 मर्ख पत्र थे जिनकी जिम्मेदारी विष्ण शर्मा को दे दी गई विष्ण शर्मा को यह पता था कि यह इतने मूर्ख हैंकि इनको प्राने तरीकों से नहीं पढ़ाया जा सकता तब उन्होंने जंत कथाओं के द्वारा पढ़ाने का निश्चय कियापंचतंत्र को पांच समूह में बनाया गया 

- Convert sentences to a DataFrame for applying lambda functions

In [5]:
sentences_p = pd.DataFrame(sentences)
sentences_p

,0
0,"पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान..."
1,आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2,चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3,"तो नहीं, उसका खून चूसने के लिए"
4,""" खेंटमल बोला, ""लेकिन मैं तम्हारा मेहमान है, म..."
...,...
81,सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82,सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83,शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84,अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...


- Add spaces around punctuation marks for each sentence

In [6]:
sentences_p[0] = sentences_p[0].apply(lambda x: re.sub(r"([!#$%&\()*+,-./:;<=>?@[\\]^_`{|}~])", r" \1 ", x))

In [7]:
sentences_p[0]

0     पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                        तो नहीं, उसका खून चूसने के लिए
4     " खेंटमल बोला, "लेकिन मैं तम्हारा मेहमान है, म...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Remove digits from the sentences

In [8]:
remove_digits = str.maketrans('', '', digits)
sentences_p[0] = sentences_p[0].apply(lambda x: x.translate(remove_digits))

In [9]:
sentences_p[0]

0     पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                        तो नहीं, उसका खून चूसने के लिए
4     " खेंटमल बोला, "लेकिन मैं तम्हारा मेहमान है, म...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Remove single quotes from the sentences

In [ ]:
sentences_p[0] = sentences_p[0].apply(lambda x: re.sub("'", '', x))

In [11]:
sentences_p[0]

0     पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                        तो नहीं, उसका खून चूसने के लिए
4     " खेंटमल बोला, "लेकिन मैं तम्हारा मेहमान है, म...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Remove double quotes from the sentences

In [12]:
sentences_p[0] = sentences_p[0].apply(lambda x: re.sub('"', '', x))

In [13]:
sentences_p[0]

0     पंचतंत्र की कहानियांपंचतंत्र नीति, कथा और कहान...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                        तो नहीं, उसका खून चूसने के लिए
4      खेंटमल बोला, लेकिन मैं तम्हारा मेहमान है, मझे...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Tokenize sentences into words using Indic NLP tokenizer \
Separate each token with a space

In [ ]:
sentences_p[0] = sentences_p[0].apply(lambda x: " ".join(indic_tokenize.trivial_tokenize(x)))

In [15]:
len(sentences_p)
sentences_p[0][:1000]

0     पंचतंत्र की कहानियांपंचतंत्र नीति , कथा और कहा...
1     आचार्य विष्ण शर्मासंस्कत के लेखक विष्णु शर्मा ...
2     चालाक खटमलएक राजा के शयनकक्ष में मंदविसर्थिणी ...
3                       तो नहीं , उसका खून चूसने के लिए
4     खेंटमल बोला , लेकिन मैं तम्हारा मेहमान है , मझ...
                            ...                        
81    सभी पशु पक्षी जब बरसात रुकने पर बाहर निकले तब ...
82    सभी का मन प्रसन्‍नता बत्तव अब झील मैं तैर रहे ...
83       शिक्षाधैर्य का फल मीठा होता हैचिड़ियाघर की सैर
84    अमन अपने मातापिता के साथ चिड़ियाघर की सैर करने...
85    अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने ...
Name: 0, Length: 86, dtype: object

- Preprocess each sentence with `<start>` and `<end>` tokens

In [16]:
def preprocess_sentence(w):
    w = w.strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [ ]:
# Skip <start>, <end> tags when not needed
def preprocess_sentence_no_tags(w):
    w = w.strip()
    return w

- Create a dataset from preprocessed sentences

In [ ]:
def create_dataset(start, end):
    sentences_preprocessed = [preprocess_sentence(l) for l in sentences_p[0][start:end]]
    #return zip(*sentence_pairs)
    return sentences_preprocessed

In [ ]:
def create_full_dataset():
    sentences_preprocessed = [preprocess_sentence(l) for l in sentences_p[0]]
    return sentences_preprocessed

- Tokenize the sentences into sequences of integers using TensorFlow's `Tokenizer` \ 
- Ensure the sequences are then padded to maintain uniform length

In [20]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

- Load the dataset

In [ ]:
def load_dataset(start, end):
    # creating cleaned input, output pairs
    inp_lang = create_dataset(start, end)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)

    return input_tensor, inp_lang_tokenizer

In [22]:
def load_full_dataset():
    # creating cleaned input, output pairs
    inp_lang = create_full_dataset()
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)

    return input_tensor, inp_lang_tokenizer

- Preview a sentence from the dataset

In [23]:
zh = create_full_dataset()
print(zh[-1])

<start> अमन ने देखा वहां छोटेछोटे बच्चे आए हैंवह अपने पैर पर चल रहे थे कोई भी अपने मम्मी पापा के गोदी में नहीं चल रहा थाइस पर अमन भी अपने छोटेछोटे पैरों से चलने लगा इस पर अमन के मम्मी पापा को बहुत खुशी हुई क्योंकिअब उसका बेटा चलना सीख रहा था <end>


- Load a subset of the dataset (sentences from index 1 to 40), and tokenize them

In [24]:
num_examples_p = 40 
input_tensor_p, inp_lang_p  = load_dataset(1, 40)

# Calculate max_length of the target tensors
max_length_inp_p = input_tensor_p.shape[1]
max_length_inp_p

137

- Creating training and validation sets using an 90-10 split

In [ ]:
input_tensor_train_p, input_tensor_val_p = train_test_split(input_tensor_p, test_size=0.1)

# Show length
len(input_tensor_train_p), len(input_tensor_val_p)

(35, 4)

In [26]:
num_examples = len(sentences_p[0])
input_tensor, inp_lang = load_full_dataset()

# Calculate max_length of the target tensors
max_length_inp = input_tensor.shape[1]
max_length_inp

188

In [27]:
input_tensor.shape

(86, 188)

In [ ]:
input_tensor_train, input_tensor_val = train_test_split(input_tensor, test_size=0.1)

# Show length
len(input_tensor_train), len(input_tensor_val)

(77, 9)

- Checking word to index mapping in the vocabulary

In [29]:
inp_lang.index_word[60]

'इस'

In [30]:
inp_lang.word_index['इस']

60

In [31]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [32]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[30])

Input Language; index to word mapping
1 ----> <start>
935 ----> सुंदरवन
11 ----> की
936 ----> कहानीसुंदरवन
235 ----> नामक
3 ----> एक
937 ----> खूबसूरत
25 ----> जंगल
6 ----> था
57 ----> वहां
66 ----> खूब
350 ----> ढ़ेर
113 ----> सारे
88 ----> जानवर
9 ----> ,
218 ----> पशु
219 ----> पक्षी
22 ----> रहा
938 ----> करते
26 ----> थे
352 ----> धीरे
352 ----> धीरे
939 ----> सुंदरवनकी
940 ----> सुंदरता
243 ----> कम
96 ----> होती
62 ----> जा
92 ----> रही
54 ----> थी
2 ----> <end>


In [33]:
input_tensor_train.shape

(77, 188)

- Create TensorFlow datasets for training and validation

In [34]:
BATCH_SIZE = 64
BUFFER_SIZE = len(input_tensor_train)

vocab_inp_size = len(inp_lang.word_index) + 1

dataset_train = tf.data.Dataset.from_tensor_slices(input_tensor_train).shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)

dataset_val = tf.data.Dataset.from_tensor_slices(input_tensor_val).shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)

- Custom data generator that can be used for batching and shuffling data in training 
- Yield batches of data (of size batch_size) for training.

In [35]:
def generator(data, min_index, max_index, shuffle=False, batch_size=128):
    if max_index is None:
        max_index = len(data) - 1
    i = min_index
    
    while True:
        if i + batch_size >= max_index:
            i = min_index
        rows = np.arange(i, min(i + batch_size, max_index))
        i += len(rows)

        # preallocate
        samples = np.zeros((len(rows), data.shape[-1])) #1st dim:rows, 2nd dim:features
        
        # fill in: For each row selected, select the number of timesteps to sample from
        for j, row in enumerate(rows):                         # for each observation (row)
            indices = rows[j]
            samples[j] = data[indices]
            
        yield samples

In [ ]:
BATCH_SIZE_P = 4
BUFFER_SIZE_P = len(input_tensor_train_p)

inp_gen_p = generator(
    input_tensor_train_p,
    min_index  = 0,
    max_index  = 39,
    batch_size = BATCH_SIZE_P
)

In [37]:
example_input_batch_p = next(inp_gen_p)
example_input_batch_p.shape

(4, 137)

In [38]:
example_input_batch_p = next(inp_gen_p)
example_input_batch_p.shape

(4, 137)

- Preview index mappings of generated input batch

In [39]:
example_input_batch_p

array([[  3.,  32.,   7., 204.,  14., 629., 630.,  79.,   4.,   5.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  3., 187., 106., 496., 106.,  16., 497.,  27.,   2., 106.,  48.,

In [ ]:
for (batch, inp) in enumerate(dataset_train):
    print(inp)
    break

tf.Tensor(
[[  1  14 190 ...   0   0   0]
 [  1   3  99 ...   0   0   0]
 [  1 188 189 ...   0   0   0]
 ...
 [  1   3  36 ...   0   0   0]
 [  1  38 218 ...   0   0   0]
 [  1 145  20 ...   0   0   0]], shape=(64, 188), dtype=int32)


- Prepare the list of sentences as a list of word lists for Word2Vec

In [42]:
hindi_sentences = [[word for word in sentence.split()] for sentence in sentences]

In [43]:
hindi_sentences

[['पंचतंत्र',
  'की',
  'कहानियांपंचतंत्र',
  'नीति,',
  'कथा',
  'और',
  'कहानियों',
  'का',
  'संग्रह',
  'है',
  'जिसके',
  'रचयिता',
  'आचार्य',
  'विष्णु',
  'शर्मा',
  'है',
  'पंचतंत्र',
  'की',
  'कहानी',
  'में',
  'बच्चोंके',
  'साथसाथ',
  'बड़े',
  'भी',
  'रुचि',
  'लेते',
  'हैं',
  'पंचतंत्र',
  'की',
  'कहानी',
  'के',
  'पीछे',
  'कोई',
  'ना',
  'कोई',
  'शिक्षा',
  'या',
  'मूल',
  'छिपा',
  'होता',
  'है',
  'जो',
  'हमें',
  'सीखदेती',
  'है',
  'पंचतंत्र',
  'की',
  'कहानी',
  'बच्चे',
  'बड़ी',
  'चाव',
  'से',
  'पढ़ते',
  'हैं',
  'तथा',
  'सीख',
  'लेते',
  'हैं',
  'पंचतंत्र',
  'की',
  'कछ',
  'कहानियां',
  'ऐसी',
  'भी',
  'है',
  'जोहिंदी',
  'में',
  'कहानी',
  'लेखन',
  'में',
  'दी',
  'जाती',
  'हैं',
  'तथा',
  'इसके',
  'साथसाथ',
  'कई',
  'परीक्षाओं',
  'में',
  'भी',
  '80780',
  '6',
  '(9',
  'कुछपाठयपृस्तक',
  'में',
  'दी',
  'होती',
  'है',
  'जो',
  'लिखने',
  'को',
  'आती',
  'हैं',
  'महत्वपूर्ण',
  'और',
  'आकर्षक',
  '40',
  'कहानियां',
  

### Create and train the Word2Vec model

In [ ]:
model = Word2Vec(
    sentences   = hindi_sentences, 
    vector_size = 100, 
    window      = 5, 
    min_count   = 1, 
    workers     = 4
)

# Save the model
model.save("models/Part1.h5")

In [ ]:
# Load the model
model = Word2Vec.load("models/Part1.h5")

# Get the vector for a specific word
vector = model.wv['उसकी']

- Preview vectors for the word 'उसकी'

In [46]:
vector

array([ 0.00420042, -0.00279677, -0.00898276, -0.00951279,  0.00636288,
       -0.00780348,  0.00626839,  0.00126905,  0.00902702,  0.0060823 ,
        0.00896631,  0.00598393, -0.00847615, -0.00956653, -0.00559273,
       -0.00142577,  0.0054648 ,  0.00217725, -0.00225392,  0.00224814,
       -0.00303053, -0.0060499 ,  0.00549394, -0.01047858,  0.00020205,
       -0.00423439,  0.00627518, -0.00493954,  0.00218526, -0.00636706,
        0.01078039, -0.00487949,  0.00395247,  0.00269254,  0.00805984,
       -0.00411804, -0.00402481, -0.00725745,  0.00525992, -0.00686732,
        0.00515348, -0.00179659,  0.00691733, -0.006217  ,  0.00089664,
       -0.00010183, -0.00998577,  0.00844939, -0.00121177,  0.0073609 ,
        0.00084453,  0.00360189,  0.00224148,  0.00903265, -0.00932341,
        0.00927403,  0.00989219,  0.00123649,  0.0034161 ,  0.00473801,
        0.00454345, -0.00267638,  0.00332576,  0.00041388,  0.00344547,
        0.00926049, -0.0078786 ,  0.00112874, -0.01063889, -0.00

- Find 10 words similar to word 'कहानी'

In [47]:
# Find similar words
similar_words = model.wv.most_similar('कहानी', topn=10)
print(similar_words)

[('गोदी', 0.3070230484008789), ('बेचने', 0.299521267414093), ("हुआ'", 0.2590577304363251), ('लेखन', 0.2559297978878021), ('रुचि', 0.25114157795906067), ('देरी', 0.24753130972385406), ('बात', 0.24468360841274261), ('अपना', 0.24308526515960693), ('केप्रार्थना', 0.24114534258842468), ('त्गे', 0.23938827216625214)]


- Show all words in the model's vocabulary

In [48]:
# Get all words in the model
words = list(model.wv.key_to_index.keys())

# Print all words
for word in words:
    print(word)

एक
में
वह
को
था
के
और
की
पर
उसने
से
है
का
नहीं
भी
गया
ने
रहा
उसे
जंगल
ही
थे
बहुत
अपने
:
मित्र
देखा
राजा
सभी
बार
पानी
तो
कि
कहा
गई
रहे
अपनी
टोपी
शेर
हो
दिन
बंदर
उसकी
कर
हैं
अमन
,
दिया
थी
जब
वहां
खूब
लिए
इस
हुए
जा
चूहा
कोई
फिर
शिक्षा
पेड़
उन्होंने
हाथी
लगा
कुछ
किसी
यह
दी
नीचे
हुआ
आ
उसके
भालू
मैं
जाता
रात
हुई
दे
आम
खून
पंचतंत्र
बुढ़िया
आया
मार
जो
जानवर
अब
सियार
शर्मा
रही
समय
टंकी
किसान
करता
सारे
दरजी
गये
उन्हें
खटमल
सकता
बच्चे
इसे
ऊपर
वहाँ
मुझे
परेशान
होती
विष्ण
तलाश
पास
छोड़
बाघ
कुत्ता
लेने
टुकड़ा
उसका
खुश
कुल्हाड़ी
देखी
साथ
"
ले
करने
प्रसन्‍न
बात
पहुँचा
जूँ
खुशी
ओर
तोड़
इसलिए
टपका
लेकिन
फल
दोनों
न
दूसरे
करना
जाने
लाया
डर
क्या
ढेर
तब
कहानी
चिड़ियाघर
मम्मी
छोटेछोटे
पापा
लड़का
अन्य
उसमें
किया
तरह
निकल
था,
तक
शयनकक्ष
सकी
पक्षी
चतुर
होता
वन
वाले
गोदी
पशु
बंडल
चले
गाँव
कहानियां
बातों
रास्ते
भाग
है"
सी
भोजन
रास्ता
आने
चूसने
तू
ऐसा
घर
इसके
चाहिए
कान
कशीदाकार
चूहे
कशीदा
तुम
है,
अपना
बाद
उन
बरसात
आगे
पेड़ों
गिर
उधर
कभी
वे
बंदरों
उस
देर
छोटे
थोड़ी
सोची
सहायता
चल
तुम्हें
जमीन
त
अंगूरों
दुकान
परछाई
